In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from app.core.config import settings
from app.core.services.intent_parser import AgentIntentParser


# Initialize Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    google_api_key=settings.google_api_key
)

# Initialize parser
parser = AgentIntentParser(llm)

# Test with a relevant query
query = "What is abhiyan working on currently?"

intent = await parser.parse_intent(query, [])
print(intent)



> Entering new AgentExecutor chain...
```json
{
  "is_relevant": true,
  "agent_intent": {
    "intent": "Find current tasks and activities for Abhiyan Timilsina across JIRA and GitHub.",
    "operations": [
      {
        "tool": "search_jira_users",
        "action": "Search for JIRA users matching 'Abhiyan Timilsina'",
        "filters": {
          "query": "abhiyan timilsina",
          "max_results": 10
        },
        "output_keys": ["jira_user_key", "jira_username", "jira_display_name"]
      },
      {
        "tool": "search_jira_issues",
        "action": "Find JIRA issues assigned to Abhiyan",
        "filters": {
          "assignee": "jira_username",
          "status": "Open",
          "max_results": 10
        },
        "output_keys": ["issue_key", "summary", "status", "project_key"]
      },
      {
        "tool": "get_github_recent_activities",
        "action": "Get Abhiyan's recent GitHub activities",
        "filters": {
          "usernames": ["abhiyan-ti

In [3]:
intent.operations

[OperationStep(tool='search_jira_users', action="Search for JIRA users matching 'Abhiyan Timilsina'", filters={'query': 'abhiyan timilsina', 'max_results': 10}, sort_by=None, order=None, output_keys=['jira_user_key', 'jira_username', 'jira_display_name'], aggregation=None),
 OperationStep(tool='search_jira_issues', action='Find JIRA issues assigned to Abhiyan', filters={'assignee': 'jira_username', 'status': 'Open', 'max_results': 10}, sort_by=None, order=None, output_keys=['issue_key', 'summary', 'status', 'project_key'], aggregation=None),
 OperationStep(tool='get_github_recent_activities', action="Get Abhiyan's recent GitHub activities", filters={'usernames': ['abhiyan-timilsina', 'abhiyantimilsina', 'atimilsina', 'abhiyan'], 'days': 7, 'include_commits': True, 'include_prs': True}, sort_by=None, order=None, output_keys=['username', 'activity_type', 'repository', 'date'], aggregation=None)]

In [1]:
from app.clients.github_client import GitHubClient

client = GitHubClient()
client.get_pull_requests()

[{'number': 4,
  'title': 'Feat/basic intent parser',
  'state': 'closed',
  'author': 'abhiyan52',
  'author_name': 'Abhiyan Timilsina',
  'created_at': '2025-09-14T09:55:04+00:00',
  'updated_at': '2025-09-14T09:55:22+00:00',
  'merged_at': '2025-09-14T09:55:22+00:00',
  'closed_at': '2025-09-14T09:55:22+00:00',
  'merged': True,
  'draft': False,
  'url': 'https://github.com/abhiyan52/team-activity-monitors/pull/4',
  'repository': 'abhiyan52/team-activity-monitors',
  'repository_full_name': 'abhiyan52/team-activity-monitors',
  'base_branch': 'main',
  'head_branch': 'feat/basic-intent-parser',
  'commits': 2,
  'additions': 351,
  'deletions': 7,
  'changed_files': 5,
  'labels': [],
  'assignees': ['abhiyan52'],
  'reviewers': [],
  'body': ''},
 {'number': 3,
  'title': 'Add testing framework and scripts for JIRA and GitHub clients',
  'state': 'closed',
  'author': 'abhiyan52',
  'author_name': 'Abhiyan Timilsina',
  'created_at': '2025-09-13T11:04:42+00:00',
  'updated_at': '